In [ ]:
from tqdm import tqdm

def comprovar_elements(llista_a, llista_b, llista_c, fitxer_sortida):
    elements_no_presentats = []
    paraules_i_index = []

    # Convertim llista_b en un conjunt per a una cerca més eficient
    set_llista_b = set(llista_b)

    # Inicialitzem la barra de progrés amb el total de la llista_a
    progress_bar = tqdm(total=len(llista_a), desc="Comparant llistes", unit=" elements")

    for index, element in enumerate(llista_a):
        if element not in set_llista_b:
            elements_no_presentats.append(index)
        progress_bar.update(1)

    progress_bar.close()

    # Creem una llista de tuples (paraula, índex) per a les paraules no presents
    paraules_i_index = [(llista_a[idx], idx) for idx in elements_no_presentats]

    with open(fitxer_sortida, 'w') as f:
        if paraules_i_index:
            for paraula, idx in paraules_i_index:
                paraula_corresponent = llista_c[idx] if idx < len(llista_c) else ''
                f.write(f"{paraula}${paraula_corresponent}\n")
        else:
            f.write("Tots els elements de la llista A estan presents a la llista B.\n")
            
def llegir_fitxer_linia_per_linia(nom_fitxer):
    with open(nom_fitxer, 'r') as fitxer:
        linies = [linia.strip() for linia in fitxer]
    return linies

def llegir_fitxer_tabulador(nom_fitxer):
    with open(nom_fitxer, 'r') as fitxer:
        linies = [linia.strip().split('\t')[1] if '\t' in linia else '' for linia in fitxer]
    return linies

fitxer1 = '../../separat/col_0.txt'  # Fitxer on cada línia conté una paraula
fitxer3 = '../../separat/col_1.txt'  # Fitxer on cada línia conté una paraula
fitxer2 = 'cawiktionary-20240601-all-titles.txt'  # Fitxer on cada línia té el format "0\tParaula"
fitxer_sortida = 'paraules_NO_presents_al_viccionari.txt'  # Fitxer on s'escriuran els resultats

llista_a = llegir_fitxer_linia_per_linia(fitxer1)
llista_b = llegir_fitxer_tabulador(fitxer2)
llista_c = llegir_fitxer_linia_per_linia(fitxer3)

comprovar_elements(llista_a, llista_b, llista_c, fitxer_sortida)

In [ ]:
import requests
from tqdm import tqdm

def llegir_fitxer(input_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            paraules = [line.split('$')[1].strip() for line in f.readlines() if line.strip()]
        return paraules
    except FileNotFoundError:
        print(f"El fitxer {input_file} no existeix.")
        return []
    except Exception as e:
        print(f"Error llegint el fitxer {input_file}: {e}")
        return []

def comprovar_pagina_diec(paraula):
    url = f'https://dlc.iec.cat/Results?DecEntradaText={paraula}&AllInfoMorf=False&OperEntrada=0&OperDef=0&OperEx=0&OperSubEntrada=0&OperAreaTematica=0&InfoMorfType=0&OperCatGram=False&AccentSen=False&CurrentPage=0&refineSearch=0&Actualitzacions=False'
    try:
        response = requests.get(url)
        response.raise_for_status()
        if "No s'ha trobat cap entrada coincident amb els criteris de cerca" in response.text:
            return False
        return True
    except requests.RequestException as e:
        print(f"Error en la petició per a la paraula {paraula}: {e}")
        return False

def escriure_resultats(output_file, resultats):
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for resultat in resultats:
                f.write(resultat + '\n')
    except Exception as e:
        print(f"Error escrivint al fitxer {output_file}: {e}")

def executar_comprovacio(input_file, output_file):
    paraules = llegir_fitxer(input_file)
    if not paraules:
        print("No hi ha paraules per processar.")
        return

    resultats = []
    
    # Inicialitzem la barra de progrés 
    progress_bar = tqdm(total=len(paraules), desc="Comparant llistes", unit=" elements")

    # Itera només sobre les línies especificades o sobre totes si num_lines és més gran que la longitud de la llista
    for i, paraula in enumerate(paraules):
        existeix = comprovar_pagina_diec(paraula)
        
        if existeix == False:
            resultats.append((paraula))

        progress_bar.update(1)

    progress_bar.close()

    escriure_resultats(output_file, resultats)


input_file = 'paraules_NO_presents_al_viccionari.txt'
output_file = 'paraules_NO_presents_al_viccionari_ni_al_DIEC.txt'

executar_comprovacio(input_file, output_file)